# <font color=#000000, style="background-color:Gray;">__[TODO]__</font>

1) Move [DBG] calls out of <font color=#000000, style="background-color:Yellow;">__MODULES AND FUNCTIONS__</font> and into <font color=#000000, style="background-color:Yellow;">__TESTING__</font>
2) Find and perform any improvements to code logic, style, organization (e.g. simpler logic, refactoring, etc.)
3) Confirm if $\texttt{results[i]}$ is computed properly in $\texttt{\textcolor{CornflowerBlue}{\_detection\_results}(...)}$
4) Clean up both calls to $\texttt{exit()}$ in $\texttt{\textcolor{CornflowerBlue}{\_update\_indices}(...)}$
5) Fix how $\texttt{indices\_err2}$ is printed by $\texttt{DBG()}$ call at the end of $\texttt{\textcolor{CornflowerBlue}{\_update\_indices}(...)}$
6) <font color=yellow>__COMPLETE__</font> $\texttt{\textcolor{CornflowerBlue}{\_init\_2nd\_check}(...)}$ so that it actually checks if_error (and rename it accordingly)
7) Check that the $\texttt{bases[i] == "iY"}$ case is using the correct operator in $\texttt{\textcolor{CornflowerBlue}{\_init\_2nd\_check}(...)}$ so that it actually checks if_error (and rename it accordingly)
8) Confirm if phase_data encoding is using the correct operator in $\texttt{\textcolor{CornflowerBlue}{\_encode\_data}(...)}$
10) Complete code for Step 5
11) Complete code for Step 6
12) Refactor everything in <font color=#000000, style="background-color:Yellow;">__TESTING__</font> into a $\texttt{qisac\_protocol(...)}$ function
13) Find and perform any improvements to code logic, style, organization (e.g. simpler logic, refactoring, etc.)
14) Perform comprehensive unit tests

# <font color=#000000, style="background-color:Yellow;">__MODULES AND FUNCTIONS__</font>

## Modules

In [1]:
# UI and debugging
from termcolor import colored

# Protocol
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Pauli
from math import ceil
import numpy as np
import random
import math
import copy


# Visualization
from qiskit.visualization import plot_circuit_layout, circuit_drawer, plot_bloch_multivector
from pylatexenc.latexencode import unicode_to_latex
from qiskit.quantum_info import Statevector

## Debug Functions

In [2]:
# Prints debug message (if _dbg_ == mode)
def DBG(msg, mode):
    global _dbg_
    if _dbg_ >= mode:
        print("\n" + colored("[DBG]", "black", "on_yellow", attrs=["bold"]) + "\t" + msg)


# Like DBG, but without icon
def LST(msg, mode):
    global _dbg_
    if _dbg_ >= mode:
        print("\n\t" +  msg)
            

# Prints error message if is_error == True; print success message otherwise
def ERR(msg_err, msg_suc, is_error):
    global _dbg_
    if _dbg_:
        if is_error:
            print("\n" + colored("[ERR]", "black", "on_red", attrs=["bold", "blink"]) + "\t" +  msg_err)
            #raise Exception("User Defined Error")
        else:
            print("\n" + colored("[ "+u'\u2713'+" ]", "black", "on_green", attrs=["bold", "blink"]) + "\t" +  msg_suc)


# New task
def NEW(msg):
    print("\n\n\n" + colored("[NEW]", "black", "on_cyan", attrs=["bold"]) + "\t" + msg)


# Used for uninitialized variables
def NIL(var="variable"):  # var is name of unitialized variable
    ERR(var + " is unitialized! Exiting...", "", True) 

## Functions

In [3]:
# Step 1.1 (compute numbers of EPR pairs needed for message, eavesdrop checks, and total)
def get_EPR_counts(message, pe, m, m_msg, m_err1, m_err2):
    # first eavesdrop pairs
    for m in range(len(message)*2):
        m_err1 = ceil((1-pe)/2*m)
        if m - m_err1 >= len(message) and (m - m_err1)%2 == 0:
            m_msg = m
            break
    m_msg = m - m_err1
    # second eavesdrop pairs
    for m in range((len(message))*2):
        m_err2 = ceil((1-pe)/2*m)
        if m - (m_err1+m_err2) >= len(message) and (m - m_err2)%2 == 0:
            m_msg = m
            break
    m_msg = m - m_err1 - m_err2
    # return counts
    DBG("m      = " + str(m) + "\n\tm_msg  = " + str(m_msg) + "\n\tm_err1 = " + str(m_err1) + "\n\tm_err2 = " + str(m_err2), 2)
    return m, m_msg, m_err1, m_err2
    

# Step 1.2 (create EPR pairs)
def create_EPR_pairs(m):
    EPR_pairs = []
    for i in range(m):
        label_A          = r"""A_"""+str(i)
        label_B          = r"""B_"""+str(i)
        q_a              = QuantumRegister(1, name=label_A)
        q_b              = QuantumRegister(1, name=label_B)
        c                = ClassicalRegister(2)
        EPR_pair_circuit = QuantumCircuit(q_a, q_b, c)
        EPR_pair_circuit.x(q_b[0])
        EPR_pair_circuit.h(q_a[0])
        EPR_pair_circuit.z(q_a[0])
        EPR_pair_circuit.z(q_b[0])
        EPR_pair_circuit.cx(q_a[0], q_b[0])
        EPR_pairs.append(EPR_pair_circuit)
    DBG(str(len(EPR_pairs)) + " EPR pairs generated", 1)
    return EPR_pairs


# Step 1.3 (choose message and error indices)
def choose_indices(m, m_msg, m_err1, m_err2, indices_msg, indices_err1, indices_err2):
    for i in range(m):
        indices_msg.append(i)
    indices_err1 = random.sample(indices_msg, m_err1)
    for i in indices_err1:
        indices_msg.remove(i)
    indices_err2 = random.sample(indices_msg, m_err2)
    for i in indices_err2:
        indices_msg.remove(i)
    return indices_msg, indices_err1, indices_err2


# Step 2
def detect_eavesdropping(simulator, _circuits, indices_err1, bases_err1, pe, threshold):
    # Get Bob's basis selections
    for i in indices_err1:
        bases_err1[i] = random.choice(['X', 'Y', 'Z'])
    # Get Alice and Bob's results
    bob_results   = _detection_results(simulator, _circuits, indices_err1, bases_err1, 1)  # 1 is Bob's qubit
    alice_results = _detection_results(simulator, _circuits, indices_err1, bases_err1, 0)  # 0 is Alice's qubit
    # Count errors
    error_count = 0
    DBG("Eavesdrop detection circuits:", 3)
    for i in indices_err1:
        LST("\n" + str(_circuits[i]) + "\n", 3)
        if alice_results[i] == bob_results[i]:
            error_count += 1
    DBG("bob_results       = " + str(bob_results) + "\n\talice_results     = " + str(alice_results), 2)
    DBG("error_count       = "+str(error_count) + "\n\tlen(indices_err1) = "+str(len(indices_err1)) + "\n\tthreshold         = "+str(threshold),2)
    is_error = (error_count / len(indices_err1)) >= threshold
    ERR("Eavesdropping (1) detected! Restarting QISAC protocol...", "No eavesdropping detected. Continuing...", is_error)
    return _circuits, indices_msg, indices_err1, bases_err1, is_error


# Step 3:
def encode(circuits, message, phase_data,m_msg, m_err2, indices_msg, indices_err2, bases_err2):
    # Check length restrictions and abord or add padding as needed
    if len(message) > len(circuits) - m_err2:
        ERR("message / phase_data lengths cannot exceed available qubits! Exiting...", "", True)
    elif len(message) < m_msg:
        for i in range(m_msg - len(message)):
            message    = "".join([message, "0"])     # [NOTE] we assume message cannot end with a 0 
            phase_data = "".join([phase_data, "0"])  # [NOTE] we assume phase_data cannot end with a 0 
        DBG("Padded message    = " + message + "\n\tPadded phase_data = " + phase_data, 1)
    circuits, bases_err2 = _init_2nd_check(circuits, indices_err2)
    circuits             = _encode_data(circuits, message, phase_data, indices_msg)    
    return circuits, message, phase_data, indices_msg, indices_err2, bases_err2


# Step 4:
def detect_eavesdropping_2(simulator, _circuits, m_msg, m_err2, indices_err2, bases_err2, pe, threshold):
    # Get Alice and Bob's results
    bob_results   = _detection_results(simulator, _circuits, indices_err2, bases_err2, 1)  # 1 is Bob's qubit
    alice_results = _detection_results(simulator, _circuits, indices_err2, bases_err2, 0)  # 0 is Alice's qubit
    # Count errors
    error_count = 0
    DBG("Eavesdrop detection circuits:", 3)
    for i in indices_err2:
        LST("\n" + str(_circuits[i]) + "\n", 3)
        if alice_results[i] == bob_results[i]:
            error_count += 1
    DBG("bob_results       = " + str(bob_results) + "\n\talice_results     = " + str(alice_results), 2)
    DBG("error_count       = "+str(error_count) + "\n\tlen(indices_err2) = "+str(len(indices_err2)) + "\n\tthreshold         = "+str(threshold),2)
    is_error = (error_count / len(indices_err2)) >= threshold
    ERR("Eavesdropping (2) detected! Restarting QISAC protocol...", "No eavesdropping detected. Continuing...", is_error) 
    return _circuits, is_error


# Step 5:
def measure_msg_pairs(simulator, circuits, indices_msg, bases_msg, po):
    results = {}
    for i in indices_msg:
        if random.random() <= po:
            circuits[i].pauli('XX', [0,1])  # [TODO] Is this correct?
            bases_msg[i] = 'O_1'
        else:
            circuits[i].pauli('YX', [0,1])  # [TODO] Is this correct?
            bases_msg[i] = 'O_2'
        # Get results
        circuits[i].measure_all()
        circuits[i] = transpile(circuits[i], simulator)
        job         = simulator.run(circuits[i], shots=1)
        result      = job.result().get_counts()
        result_str  = list(result.keys())[0]
        results[i]  = result_str  # [TODO] Confirm this is correct
    return results, bases_msg

## Utility Functions

In [5]:
# Helper function for Step 2
def _detection_results(simulator, circuits, indices, bases, qubit):
    results = {}
    for i in indices:
        # Prepare circuits at indices
        if   bases[i] == 'X':
            circuits[i].h(qubit)
        elif bases[i] == 'Y':
            circuits[i].sdg(qubit)
            circuits[i].h(qubit)
        # Get results
        circuits[i].measure(qubit, qubit)
        circuits[i] = transpile(circuits[i], simulator)
        job         = simulator.run(circuits[i], shots=1)
        result      = job.result().get_counts()
        result_str  = list(result.keys())[0]
        results[i]  = result_str[qubit]  # [TODO] Confirm this is correct
    return results


# Helper function for Step 3
def _init_2nd_check(circuits, indices_err2):
    bases = {}
    for i in indices_err2:
        bases[i] = random.choice(["I", "Z", "X", "iY"])
        if   bases[i] == "Z":
            circuits[i].z(0)
        elif bases[i] == "X":
            circuits[i].x(0)
        elif bases[i] == "iY":
            circuits[i].ry(np.pi/2, 0)  # [TODO] Is this the correct operator?
    DBG("bases          =  " + str(bases), 2)
    return circuits, bases

# Helper function for Step 3
def _encode_data(circuits, message, phase_data, indices_msg):
    # Encode message
    DBG("Encoding message", 1)
    j = 0
    for i in indices_msg:
        if message[j] == "1":
            circuits[i].x(0)
        j += 1
    # Encode phase_data
    for i in indices_msg:
        circuits[i].rz(math.radians(int(phase_data)), 0) # [TODO] Is this the correct operator?
    DBG("Encoded message/parameter and 2nd eavesdrop check circuits:", 3)
    for c in circuits:
        LST("\n" + str(c), 3)
    return circuits

def _load_sensor_data(file_path):

    adc1_values = []

    with open(file_path, 'r') as file:
        for line in file:
            if 'ADC1 =' in line:
                parts = line.strip().split()
                adc1_values.append(float(parts[2]))

    arr = np.array(adc1_values, dtype=np.float32)
    return arr

# <font color=#000000, style="background-color:Yellow;">__QISAC TESTING__</font>

<font color=#000000, style="background-color:Yellow;">__[ ! ]__</font> $\hspace{0.2mm}$ Code in this section can be rolled into a $\texttt{qisac()}$ function with all global variables as parameters for "production" usage

In [8]:
# Global variable declarations and initializations
_dbg_        = 2                    # If _dbg_, then print additional debug statements
m            = None                 # Number of EPR pairs
m_msg        = None                 # Number of EPR pairs allocated to message / phase accumulation
m_err1       = None                 # Number of EPR pairs allocated to first eavesdropping check
m_err2       = None                 # Number of EPR pairs allocated to second eavesdropping check
indices_msg  = []                   # Indices from circuits allocated to message / phase accumulation
indices_err1 = []                   # Indices from circuits allocated to first eavesdropping check (won't be meaningful after Step 2)
indices_err2 = []                   # Indices from circuits allocated to second eavesdropping check
pe           = 0.8                  # Proportion of EPR pairs NOT used for 1st error detection
po           = 0.5                  # Proportion of EPR pairs measured on observable 1 (see step 5)
threshold    = 0.7                  # Threshold for first eavesdropping detection
simulator    = AerSimulator()       # Create simulator
circuits     = None                 # Used to store list of EPR pairs
is_error     = True                 # Used to indicate if eavesdropping was detected (see Step 2)
message      = ""                   # Message to be communicated
phase_data   = ""                   # Phase data to be accumulated
N            = 1                    # Number of evolution iterations of phase accumulation
bases_err1   = {}                   # Bases selected by Bob for 1st eavesdropping check
bases_err2   = {}                   # Bases selected by Alice for 2nd eavesdropping check
bases_msg    = {}
#O_1          = Pauli('XX')          # Obersable 1 (see Step 5)
#O_2          = Pauli('YX')          # Obersable 2 (see Step 5)


while is_error:
    message      = "00110111011001011101110100011001"  # multiply sensor data by 10^d to remove fractional part
    phase_data   = "00000000000000000000000000000000"  
    indices_msg  = []
    indices_err1 = []
    indices_err2 = []

    print("\n" + colored("[STARTING QISAC PROTOCOL]", "black", "on_magenta", attrs=["bold"]))
    
    NEW("(1) Prepare list of EPR pairs and send one qubit from each to Bob")
    m, m_msg, m_err1, m_err2                = get_EPR_counts(message, pe, m, m_msg, m_err1, m_err2)
    circuits                                = create_EPR_pairs(m)  # Create list of EPR pair circuits
    indices_msg, indices_err1, indices_err2 = choose_indices(m, m_msg, m_err1, m_err2, indices_msg, indices_err1, indices_err2)
    DBG("indices_msg  = " + str(indices_msg) + "\n\tindices_err1 = " + str(indices_err1) + "\n\tindices_err2 = " + str(indices_err2), 2)


    NEW("(2) Check for eavesdropping")
    # add extra param to detect_eavesdropping() as either True or False to debug
    circuits, indices_msg, indices_err1, bases_err1, is_error = detect_eavesdropping(simulator, circuits, indices_err1, bases_err1, pe, threshold)  

    if is_error:
        print("\n")  # UI formatting
        continue
    
    NEW("(3) Encode message and parameter estimate")
    circuits, message, phase_data, indices_msg, indices_err2, bases_err2 = encode(circuits, message, phase_data, m_msg, m_err2, indices_msg, indices_err2, bases_err2)


    NEW("(4) Transmit remaining qubits and check again for eavesdropping")
    cicruits, is_error = detect_eavesdropping_2(simulator, circuits, m, m_err2, indices_err2, bases_err2, pe, threshold)

    if is_error:
        print("\n")  # UI formatting
        continue
        

NEW("(5) Perform measurements")
results, bases_msg = measure_msg_pairs(simulator, circuits, indices_msg, bases_msg, po)
DBG("Measurement results:", 1)
msg_check = ""
for q,v in results.items():
    if v[0] == v[1]:
        msg_check = msg_check + str(1)
    else:
        msg_check = msg_check + str(0)
DBG("msg_check = " + msg_check + "\n\tmessage   = " + message, 1)
ERR("Decoded message does not match sent message!", "Decoded message correctly!", msg_check != message)


#if msg_check == message:
#    NEW("(6) Estimate parameter")
#    def estimate_parameter(circuits, indices_msg, ):
#        return parameter

#    parameter = estimate_parameter




[STARTING QISAC PROTOCOL]



[NEW]	(1) Prepare list of EPR pairs and send one qubit from each to Bob

[DBG]	m      = 40
	m_msg  = 32
	m_err1 = 4
	m_err2 = 4

[DBG]	40 EPR pairs generated

[DBG]	indices_msg  = [0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 27, 28, 29, 31, 32, 33, 34, 35, 38, 39]
	indices_err1 = [11, 7, 30, 22]
	indices_err2 = [3, 26, 37, 36]



[NEW]	(2) Check for eavesdropping

[DBG]	bob_results       = {11: '0', 7: '0', 30: '0', 22: '0'}
	alice_results     = {11: '0', 7: '1', 30: '1', 22: '0'}

[DBG]	error_count       = 2
	len(indices_err1) = 4
	threshold         = 0.7

[ ✓ ]	No eavesdropping detected. Continuing...



[NEW]	(3) Encode message and parameter estimate

[DBG]	bases          =  {3: 'X', 26: 'I', 37: 'I', 36: 'Z'}

[DBG]	Encoding message



[NEW]	(4) Transmit remaining qubits and check again for eavesdropping

[DBG]	bob_results       = {3: '0', 26: '0', 37: '0', 36: '0'}
	alice_results     = {3: '0', 26: '0', 37: '1', 36: